In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from pathlib import Path
from glob import glob
from tqdm.auto import tqdm
import numpy as np
from scipy.spatial import KDTree
from matplotlib import pyplot as plt
import matplotlib
import tiledb
from tqdm.auto import tqdm
import geopandas as gpd

In [3]:
from pyspatialkit.storage.geostorage import GeoStorage
from pyspatialkit.dataobjects.geoshape import GeoShape
from pyspatialkit.dataobjects.georaster import GeoRaster
from pyspatialkit.dataobjects.geopointcloud import GeoPointCloud
from pyspatialkit.spacedescriptors.georect import GeoRect
from pyspatialkit.crs.geocrs import GeoCrs
from pyspatialkit.spacedescriptors.geobox2d import GeoBox2d
from pyspatialkit.visualization.cesium.backend.server import start_server

[Warning] Since Open3D 0.15, installing Open3D via conda is deprecated. Please re-install Open3D via: `pip install open3d -U`.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
DATA_PATH = Path('/hpi/fs00/home/tobias.pietz/pyspatialkit/data/')
TMP_DATA_PATH = Path('/tmp/pyspatialkit/data/')
GEOSTORAGE_PATH = TMP_DATA_PATH / 'geostore/'
CRS = GeoCrs.from_epsg(25832)
AOI = GeoShape.from_shapefile(DATA_PATH / 'aoi.shp').to_crs(CRS)
AOI_BOUNDS = AOI.bounds

In [6]:
storage = GeoStorage(GEOSTORAGE_PATH)

extent:[ 71271.3483359   43257.17673566 120000.        ]
extent:[ 71271.3483359   43257.17673566 120000.        ]


In [17]:
storage.layers

{'rgbi': <pyspatialkit.storage.raster.georasterlayer.GeoRasterLayer at 0x7f4f0465c250>,
 'dom': <pyspatialkit.storage.pointcloud.geopointcloudlayer.GeoPointCloudLayer at 0x7f4f3c6d0750>,
 'dgm': <pyspatialkit.storage.pointcloud.geopointcloudlayer.GeoPointCloudLayer at 0x7f4f04541e50>}

In [8]:
rgbi_layer = storage.get_layer('rgbi')
dom_layer = storage.get_layer('dom')
dgm_layer = storage.get_layer('dgm')

In [12]:
test_bounds = (722830.9, 5747402.9, 723030.6, 5747608.7)
test_box = GeoBox2d.from_bounds(test_bounds, crs=CRS)
test_rect = test_box.to_georect()

In [13]:
img = rgbi_layer.get_data(test_rect)

In [16]:
start_server(storage)

STARTING SERVER


RuntimeError: Directory '/workspaces/pyspatialkit/frontend/spatialkitcesium/public' does not exist